In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
# Load methods to extract data from rosbags
from unpack_rosbag import unpack_bag, synchronize_topics
from imu_ramp_offline import ImuRampDetect
from tf.transformations import euler_from_quaternion
from scipy.signal import resample

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [ ]:
#! Select a bag
BAG_NUM = -2

BAG_NAMES = [
    "d_d2r2s_lidar_wo_odom_hdl.bag",
    "d_d2r2s_odom_hdl.bag",
    "d_e2q_hdl.bag",
    "straight_wo_ramps_odom_hdl.bag",
    "u_c2s_half_odom_hdl.bag",
    'u_c2s_half_odom_stereo_hdl.bag',
    "u_c2s_hdl.bag",
    "u_c2s_stop_hdl.bag",
    "u_d2e_hdl.bag",
    "u_s2c_half_odom_hdl.bag",
    "u_s2c2d_hdl.bag"
]

# ROS topics
ODOM_TOPIC = '/eGolf/sensors/odometry'
IMU_TOPIC = '/zed2i/zed_node/imu/data'
# IMU_TOPIC = '/imu/data'

# Rosbag path
BAG_PATH = "/home/user/rosbags/final/slam/" + BAG_NAMES[BAG_NUM]

Loading some functions

In [ ]:
# Accelerometer
def pitch_from_acc(acc_x_imu):
    return np.rad2deg(np.arcsin(acc_x_imu / g_mag))

# Odometer
def vel_from_odom(odom, track_width = 1.52):
    """Car velocity from wheel speeds"""
    alpha = (odom[:, 2] - odom[:, 3]) / track_width
    yaw = alpha * 1 / f
    # 3.6 to convert from km/h to m/s
    vel_x_car = ((odom[:, 2] + odom[:, 3]) / 2) * np.cos(yaw) / 3.6
    return vel_x_car

def acc_from_vel(sig):
    return np.diff(sig, prepend=0) / (1.0 / f)

def pitch_from_grav(acc_x_imu, odom, win):
    # Calculate car velocity
    vel_x_car = vel_from_odom(odom)
    # Calculate acceleration of filtered velocity
    acc_x_car_filt = acc_from_vel(moving_average(vel_x_car, win))
    acc_x_imu_filt = moving_average(acc_x_imu, win)
    acc_x_fuse = acc_x_imu_filt - acc_x_car_filt
    return np.rad2deg(np.arcsin(acc_x_fuse / g_mag))

# Gyroscope
def pitch_from_gyr(ang_vel_y):
    return -np.rad2deg(np.cumsum(ang_vel_y) / f)

def remove_drift(ang_vel):
    drift = np.zeros(len(ang_vel))
    # Only remove drift if end position is also on flat ground
    if not "half" in BAG_PATH:
        drift = np.linspace(ang_vel[0], ang_vel[-1], len(ang_vel))
    ang_y_no_drift = ang_vel - drift
    return ang_y_no_drift, drift

# Complementary filter
def complementary_filter(acc_x_imu, vel_y_imu, f, K):
    # Delay
    # K = tau / (tau + 1.0/f)
    # initial estimations
    angle_k = 0
    angle_est = []
    for i, v in enumerate(acc_x_imu):
        # Calculate pitch angle using linear acceleration data
        theta_a = np.arcsin(acc_x_imu[i] / g_mag)
        angle_k = K * (angle_k - vel_y_imu[i] / f) + (1 - K) * theta_a
        # Ignore acc readings if they are unreliable
        angle_est.append(angle_k)
    return np.rad2deg(angle_est)

def complementary_filter_grav(acc_x_imu, vel_y_imu, odom, f, K):
    # Delay
    # K = tau / (tau + 1.0/f)
    # initial estimations
    angle_k = 0
    angle_est = []
    theta_a = np.deg2rad(pitch_from_grav(acc_x_imu, odom, 0.5))
    for i, v in enumerate(acc_x_imu):
        # Calculate pitch angle using linear acceleration data
        angle_k = K * (angle_k - vel_y_imu[i] / f) + (1 - K) * theta_a[i]
        # Ignore acc readings if they are unreliable
        angle_est.append(angle_k)
    return np.rad2deg(angle_est)

def get_compl_gain(T_free):
    f0 = 1.0 / T_free
    alpha = 1 / (2*np.pi*f0)
    gamma = alpha / (alpha + 1.0 / f)
    return gamma

def get_gyro_trust(gamma):
    return 1/(1/gamma/(2*np.pi))

# Lidar
def pitch_from_lidar(odom_hdl):
    euler_angles = []
    for i, ori in enumerate(odom_hdl):
        q = [ori.orientation.x,
            ori.orientation.y,
            ori.orientation.z,
            ori.orientation.w]
        euler_angle = euler_from_quaternion(q)
        euler_angles.append(euler_angle)
    euler_angles = -np.rad2deg(np.asarray(euler_angles))
    t = np.linspace(0,odom.shape[0]/f, len(odom_hdl))
    # Only return pitch angle
    return euler_angles[:, 1], t

def upsample(sig, size):
    # Get the flipped signal
    flip = np.flip(sig)
    # Mirror signal at the beginning at start and end
    sig_extended = np.hstack((flip, sig, flip))
    # Resample signal (*3 due to adding two flipped parts)
    sig_ext_up = resample(sig_extended, size * 3)
    # Get original signal again
    sig_up = np.split(sig_ext_up, 3)[1]
    return sig_up

# Ramp properties
def ramp_properties(dist, angle):
    # Start index of ramp
    start_idx = np.flatnonzero(dist)[0]
    # End index of ramp
    # TODO: Instead of using last val use first value after change is near constant (angle has settled)
    try: 
        end_idx = start_idx + np.flatnonzero(dist[start_idx:] == 0)[0] - 1
    except IndexError:
        end_idx = len(dist) - 1
    # Length of ramp
    length = dist[end_idx]
    # Angle of ramp
    angle = np.mean(angle[start_idx: end_idx])
    print("Length of ramp: {:.2f}m\nAngle of ramp: {:.2f} deg".format(length, angle))
    
def ramp_detect_time(dist):
    return np.flatnonzero(dist)[0] / float(f)

# Filter
def moving_average(sig, win):
    # Convert window size from s to idx
    win = int(win * f)
    b = np.ones(win) / win
    a = np.ones(1)
    return signal.lfilter(b, a, sig)

# Evaluation
def RMSE(y, y_pred):
    return np.sqrt(((y_pred - y) ** 2)).mean()

#### Importing the data

In [ ]:
# IMU data (angular velocity and linear acceleration)
imu, t_imu = unpack_bag(BAG_PATH, IMU_TOPIC, 'imu')
quat, _ = unpack_bag(BAG_PATH, IMU_TOPIC, 'quat')
# Odom data (wheel speeds and handwheel angle)
odom, t_odom = unpack_bag(BAG_PATH, ODOM_TOPIC, 'car_odom')
# Odom data from hdl slam
odom_hdl, t_odom_hdl = unpack_bag(BAG_PATH, "/odom",'hdl_odom')

# Ignore odom if none has been recorded
if len(odom) == 0:
    odom = np.zeros((len(imu), 5))
    is_odom = False
else:
    # Synchronize both topics (IMU is 400 Hz whereas odom is only 100 Hz)
    # Downsamples higher freq signal and matches start and end time
    imu, odom = synchronize_topics(imu, t_imu, odom, t_odom)
    is_odom = True
    
# Split linear acceleration and angular velocity 
ang_vel = imu[:, 0]
lin_acc = imu[:, 1]
# Magnitude of accelerometer when still
g_mag = np.linalg.norm(np.mean(lin_acc[:100], axis=0))

# Frequency (of slowest sensor) [Hz]
f = 400
if is_odom or IMU_TOPIC == '/imu/data':
    f = 100
print(f)
# Time vector
t = np.linspace(0, odom.shape[0]/f, odom.shape[0])

### Run the algorithm
Returns estimated angles and raw data of imu in car frame

In [ ]:
ird = ImuRampDetect(f)
tf, gyr_bias = ird.align_imu(lin_acc, ang_vel, odom)
all_angles = []
lin_acc_car = []
ang_vel_car = []
dist = []
for i, _ in enumerate(lin_acc):
    la, av, angles, d = ird.spin(lin_acc[i], ang_vel[i], odom[i], tf, gyr_bias)
    lin_acc_car.append(la)
    ang_vel_car.append(av)
    all_angles.append(angles)
    dist.append(d)
    
# Convert to numpy array
lin_acc_car = np.asarray(lin_acc_car)
ang_vel_car = np.asarray(ang_vel_car)
dist = np.asarray(dist)
# Convert from radians to degree
angles = np.rad2deg(np.asarray(all_angles))
# Convert to pandas dataframe (for plotting)
df_angles = pd.DataFrame(angles, columns = ["acc", "gyr", "acc_odom", "compl", "compl_grav"], index=t)

# Only linear acceleration measured along x-axis is of interest
acc_x_imu = lin_acc_car[:, 0]
# Only angular velocity measured along y-axis is of interest
vel_y_imu = ang_vel_car[:, 1]

print("First ramp contact after {:.2f} s".format(ramp_detect_time(dist)))
ramp_properties(dist, angles[:, -1])

Linear acceleration only and with odom

In [ ]:
# Car pitch angle using raw accelerometer data
car_angle_acc = pitch_from_acc(acc_x_imu)
# Car pitch angle using filtered accelerometer data
car_angle_acc_filt = pitch_from_acc(moving_average(acc_x_imu, 0.5))

# Car pitch angle using accelerometer data with acc from odom subtracted
car_angle_grav = pitch_from_grav(acc_x_imu, odom, 0.5)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y = car_angle_acc, name="acc"))
fig.add_trace(go.Scatter(x=t, y = car_angle_acc_filt, name="acc_filt"))
fig.add_trace(go.Scatter(x=t, y = car_angle_grav, name="grav"))

Angular velocity only

In [ ]:
# Car pitch angle using raw gyroscope data
car_angle_gyr = pitch_from_gyr(vel_y_imu)
# Removing drift from gyroscope (only works if end position is on flat ground again)
car_angle_gyr_no_drift, drift = remove_drift(car_angle_gyr)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y = car_angle_gyr, name="gyr"))
fig.add_trace(go.Scatter(x=t, y = car_angle_gyr_no_drift, name="gyr_no_drift"))
# fig.add_trace(go.Scatter(x=t, y = drift, name="drift"))

Complementary filter

The time $T_\mathrm{free}$ where the drift of the angle estimation from the gyroscope is negligible, is inversely proportional to the cut-off frequency $f_0 = \frac{1}{T_\mathrm{free}}$.
E.g. if the drift from the gyroscope is only negligible for 10 s then the cut-off frequency should be chosen at $f_0 ~= ~\frac{1}{10 s} = 0.1 Hz$ or higher.
The value of the filter gain $\gamma$ can then be calculated using 
\begin{equation}
	\alpha
	= \frac{\tau}{\tau + T}
	% = \frac{1}{w_0}
	= \frac{1}{2\pi f_0}
\end{equation}
with $T=\frac{1}{f_s}$
\begin{equation}
	\gamma = \frac{\alpha}{\alpha + T} \in [0,1]
\end{equation}
The complementary filter formula is
\begin{equation}
	\hat{\theta}_t = \gamma\left(\hat{\theta}_{t - 1} + T \omega_{\mathrm{gyr},t}\right) + (1 - \gamma) \theta_{\mathrm{acc},t},
\end{equation}

**TODO**: 

Complementary filter which fuses angle from gravity method with gyroscope.
Problem is that the angle from gravity method is delayed, which is not easy to implement for the angle calculation using angular velocity



In [ ]:
def ramp_angle_est(angle, win):
    """Get estimated (final ramp angle)"""
    wait_for_drop = False
    ang_buffer = []
    settled = []
    for i,a in enumerate(angle):
        ang_buffer.append(a)
        if len(ang_buffer) > win:
            ang_buffer.pop(0)
        # Derivate signal (add div by t)
        c = ang_buffer[-1] - ang_buffer[0]
        # Check if change is big and angle is above thresh
        if np.abs(a) > 1 and c > 0.05:
            wait_for_drop = True
        if wait_for_drop:
            if c < 0.02:
                settled.append(a)
                if len(settled) == win:
                    diff = settled[-1] - settled[0]
                    if diff < 0.3:
                        return np.mean(settled)

In [ ]:
# Car pitch angle using raw accelerometer data
car_angle_grav = pitch_from_grav(acc_x_imu, odom, 0.5)
# Car pitch angle using raw gyroscope data
car_angle_gyr = moving_average(pitch_from_gyr(vel_y_imu), 0.5)

gamma1, gamma2 = 0.9998, 0.99
car_angle_compl = complementary_filter(acc_x_imu, vel_y_imu, f, gamma1)
car_angle_compl2 = complementary_filter(acc_x_imu, vel_y_imu, f, gamma2)
car_angle_compl_grav = complementary_filter_grav(acc_x_imu, vel_y_imu, odom, f, gamma2)
print("Filter 1 assumes that gyro is reliable for {:.3f} s and filter 2 for {}".format(
    get_gyro_trust(gamma1), get_gyro_trust(gamma2)
))
# Calculate pitch angle from lidar hdl slam
car_angle_lidar, t2 = pitch_from_lidar(odom_hdl)
car_angle_lidar_up = upsample(car_angle_lidar, len(car_angle_acc))

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=car_angle_acc, name="acc"))
fig.add_trace(go.Scatter(x=t, y=car_angle_gyr, name="gyr"))
fig.add_trace(go.Scatter(x=t, y=car_angle_compl, name="compl 0.9998"))
fig.add_trace(go.Scatter(x=t, y=car_angle_compl2, name="compl 0.99"))
fig.add_trace(go.Scatter(x=t, y=car_angle_compl_grav, name="compl grav 0.99"))
fig.add_trace(go.Scatter(x=t2, y=car_angle_lidar, name="lidar"))
fig.add_trace(go.Scatter(x=t, y=car_angle_lidar_up, name="lidar_upsampled"))


In [ ]:
pio.templates.default = "plotly_white"

# def plot_for_latex(self, x=None, y=None, mode="lines+markers"):
"""Layout template for exporting plots to latex"""
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=car_angle_acc, name=r'$\theta_\mathrm{acc}$'))
fig.add_trace(go.Scatter(x=t, y=car_angle_gyr, name=r"$\theta_\mathrm{gyr}$"))
# fig.add_trace(go.Scatter(x=t, y=car_angle_grav, name=r"$\theta_\mathrm{grav}$"))
fig.add_trace(go.Scatter(x=t, y=car_angle_compl2, name=r"$\hat{\theta}$"))
# fig.add_trace(go.Scatter(x=t, y=car_angle_compl_grav, name=r"$\hat{\theta}_\mathrm{grav}$"))
fig.update_xaxes(
    title_text="Time [s]",
    showline=True,
    linewidth=1,
    linecolor="black",
    mirror=True,
)
fig.update_yaxes(
    title_text="Angle [deg]",
    showline=True,
    linewidth=1,
    linecolor="black",
    mirror=True,
)
fig.update_layout(
    width=600,
    height=300,
    font_family="Serif",
    font_size=14,
    font_color="black",
    margin_l=5,
    margin_t=5,
    margin_b=5,
    margin_r=5,
    title="",
    legend=dict(x=0.01, y=0.01, traceorder="normal", bordercolor="Gray", borderwidth=1),
)
# fig.write_image("test.pdf")
# fig.show()
# return fig

Overview of all angle calculation methods and the measured ramp distance

In [ ]:
fig = px.line(df_angles)
fig.update_layout(xaxis_title='Time [s]', yaxis_title='Angle [deg]', title=BAG_NAMES[BAG_NUM])
fig.show()

fig = px.line(dist)
fig.update_layout(xaxis_title='Time [s]', yaxis_title='Distance [m]', title=BAG_NAMES[BAG_NUM])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=car_angle_acc))
fig.add_trace(go.Scatter(x=t, y=car_angle_gyr))
# fig.add_trace(go.Scatter(y=ang_from_compl))
# Add multiple compl opt
step_vals = np.arange(1, 0.98, -0.0001)
# Add traces, one for each slider step
for step in step_vals:
    fig.add_trace(go.Scatter(visible=False, x=t, y=complementary_filter(acc_x_imu[:7100], vel_y_imu[:7100], f, step)))
step_vals_label = np.insert(step_vals, 0,  [np.nan]*2)
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] * 2 + [False] * len(fig.data)},
              {"title": "Angle using complementary filter with alpha: {:.4f}\n(assumes drift is negligible for {}s".format(
                  step_vals_label[i], 1/(1/step_vals_label[i]/(2*np.pi)))}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(active=0, currentvalue={"prefix": "alpha: "}, steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()

Orientation from IMU

In [ ]:
from scipy.spatial.transform import Rotation
# Quaternions calculated by imu
rot_imu = Rotation.from_quat(quat)
# Transformation from imu to car frame
rot = Rotation.from_dcm(np.inner(rot_imu.as_dcm(), tf))

px.line(rot.as_euler("xyz", True))

In [ ]:
def synchronize_topics42(*array):
    # Split into time and data
    array_data = np.array([ar[0] for ar in array])
    array_t = np.array([ar[1] for ar in array])
    # # Put all vectors into an array
    # arrays = np.array([ar for ar in array])
    # Adapt start and end time
    array_data_synch, array_t_synch = correct_time_diff(array_data, array_t)
    # Upsample arrays of lower frequency
    return array_data_synch
    array_match, t = upsample(array_data_synch, array_t_synch)
    return array_match, t

def correct_time_diff(array_data, array_t):
    # Find latest starting time
    t_start = max([lst[0] for lst in array_t])
    # Find earliest end time
    t_end = min([lst[-1] for lst in array_t])
    
    # Find index closest to starting / end time
    diff_start = np.abs(array_t - t_start)
    diff_end = np.abs(array_t - t_end)
    start_idx = [np.argmin(i) for i in diff_start]
    end_idx = [np.argmin(i) + 1 for i in diff_end]
    
    # Cut start / end of array to match time of array with latest / earliest start / end
    array_data_synch = [array_data[i][start_idx[i]:end_idx[i]] for i in range(len(array_data))]
    array_t_synch = [array_t[i][start_idx[i]:end_idx[i]] for i in range(len(array_t))]
    return array_data_synch, array_t_synch

def upsample(array_data, array_t):
    # Find biggest array
    biggest_idx = np.argmax([len(lst) for lst in array_t])
    print(len(array_data[biggest_idx]))
    for i,v in enumerate(array_t):
        if i != biggest_idx:
            array_data[i] = resample(array_data[i], len(array_data[biggest_idx]))
    # Time vector is the one of the biggest array
    t = array_t[biggest_idx]
    return array_data, t

# IMU data (angular velocity and linear acceleration)
imu = unpack_bag(BAG_PATH, IMU_TOPIC, 'imu')
quat, _ = unpack_bag(BAG_PATH, IMU_TOPIC, 'quat')
# Odom data (wheel speeds and handwheel angle)
odom = unpack_bag(BAG_PATH, ODOM_TOPIC, 'car_odom')
# Odom data from hdl slam
odom_hdl = unpack_bag(BAG_PATH, "/odom",'hdl_odom')
